In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import json
from sqlalchemy import create_engine, inspect, Table, Column
import pandas as pd
from ilxutils.args_reader import read_args
from ilxutils.scicrunch_client import scicrunch
import sys
import time
from ilxutils.interlex_sql import interlex_sql
import re
from collections import defaultdict
from IPython.core.display import display
import subprocess as sb

args = read_args(api_key='../keys/production_api_scicrunch_key.txt', db_url='../keys/production_engine_scicrunch_key.txt', production=True)
sql = interlex_sql(engine_key=args.db_url)
sci = scicrunch(api_key=args.api_key, base_path=args.base_path, engine_key=args.db_url)

In [ ]:
ex = sql.get_existing_ids()

In [ ]:
terms = sql.get_terms()

In [ ]:
ex

In [ ]:
broken_ex = json.load(open('../dump/broken_exisiting_ids_may_1_2018.json', 'r'))

In [ ]:
i = 18
iri = broken_ex[i][1]
if iri == broken_ex[i+1][1] or iri == broken_ex[i-1][1]:
    print('*** DUP ***')
print(broken_ex[i][0])
def curr_ex(iri):
    multi = ex.loc[ex.iri == iri]
    print('==='*20 + ' Total ' + '==='*20)
    display(multi)
    for i, row in multi.iterrows():
        print('\n'+'==='*20 + ' ' + str(i) + ' ' + '==='*20)
        display(ex.loc[ex.tid == row.tid])
        display(terms.loc[terms.id == row.tid])
        grep = !grep -nir $row.curie /home/troy/Desktop/work/NIF-Ontology/
        if grep:
            print(grep)
            
curr_ex(iri)

In [ ]:
!python3 /home/troy/Desktop/my_tools/ControlF4Folder.py -t .ttl -f /home/troy/Desktop/work/NIF-Ontology/ -s 'birnlex:656' 

In [ ]:

fixes = [
{'':''},# DICOM [0,1] ilx_0100264 ilx_0100265 same dicom but ilx_0100265 was supposed to have 0072 bc it doesnt exist and 0073 does
{'ilx_0109644':'NCBITaxon_8397'},#[2] ilx_0100800 is correct but ilx_0109644 is not and needs to be deleted... NIF-Organism.ttl points to anura from NIF-Eagle-1-Bridge.ttl
{'':''},# DICOM [3] ilx_0100879 is subclassof tiger moths so im pretty sure the child does not own this iri
{'ilx_0106754':'NCBITaxon_9773'},#[4] ref is currect, target is wrong ilx_0106754 is NCBITaxon_9773
{'ilx_0101274':'NCBITaxon_29078'},# [5] Big Brown Bat is wrong
{'':''},# [6] DICOM
{'':''},# [7] DICOM
{'ilx_0101385':'NCBITaxon_7090'}, #[8, 9] Bombyx needs to be corrected
{'ilx_0101619':'NCBITaxon_6753'}, #[10, 11]
{'':''},# [12] DICOM
{'':''},# [13] same as [5] is already handled
{'':''},# [14, 15] DICOM
{'':''},# [16]
{'':''},# [17] ITISTSN:78156
{'ilx_0106722':'NCBITaxon_6421'},# [18]
{'':''},
{'':''},
{'':''},
{'':''},
{'':''},
{'':''},
{'':''},
{'':''},
{'':''},
{'':''},
{'':''},
{'':''},
{'':''},
{'':''},
{'':''},
{'':''},
{'':''},
{'':''},
{'':''},
{'':''},
{'':''},
{'':''},
{'':''},
{'':''},
]